In [2]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import json
import re
import csv
from io import StringIO
from datetime import datetime

In [3]:
Transactions = pd.read_csv('Transactions.csv')
Transactions = Transactions.drop(['ISIN','Miejsce wykonania','Identyfikator zlecenia'],axis=1)
Transactions = Transactions.rename({'Unnamed: 8':'waluta_pozycji','Unnamed: 10':'waluta_wartości_lok','Unnamed: 12':'waluta_zak','Unnamed: 15':'Opł_trans_waluta','Unnamed: 17':'waluta_zak_razem'}, axis= 1)
Transactions['Czas'] = Transactions['Czas'].astype('datetime64').dt.time

In [4]:
Transactions

,Data,Czas,Produkt,Giełda referenc,Liczba,Kurs,waluta_pozycji,Wartość lokalna,waluta_wartości_lok,Wartość,waluta_zak,Kurs wymian,Opłata transakcyjna,Opł_trans_waluta,Razem,waluta_zak_razem
0,02-02-2022,17:31:00,PAYPAL HOLDINGS INC.,NDQ,1,131.750,USD,-131.75,USD,-116.63,EUR,1.1296,-0.50,EUR,-117.13,EUR
1,04-01-2022,20:37:00,APPLE INC. - COMMON ST,NDQ,1,179.800,USD,-179.80,USD,-159.24,EUR,1.1291,-0.50,EUR,-159.74,EUR
2,04-01-2022,20:36:00,PAYPAL HOLDINGS INC.,NDQ,1,190.380,USD,-190.38,USD,-168.61,EUR,1.1291,-0.50,EUR,-169.11,EUR
3,06-12-2021,16:10:00,MICROSOFT CORPORATION,NDQ,1,319.870,USD,-319.87,USD,-283.42,EUR,1.1286,NaN,NaN,-283.42,EUR
4,16-11-2021,16:45:00,NVIDIA CORPORATION - C,NDQ,1,299.710,USD,-299.71,USD,-264.22,EUR,1.1343,-0.50,EUR,-264.72,EUR
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78,31-03-2020,14:01:00,ISHRC S&P 500,LSE,1,260.000,USD,-260.00,USD,-237.33,EUR,1.0955,-2.07,EUR,-239.40,EUR
79,30-03-2020,15:30:00,DELTA AIR LINES INC.,NSY,1,28.110,USD,-28.11,USD,-25.49,EUR,1.1028,-0.50,EUR,-25.99,EUR
80,26-03-2020,17:38:00,DELTA AIR LINES INC.,NSY,1,35.000,USD,-35.00,USD,-31.75,EUR,1.1025,-0.50,EUR,-32.25,EUR
81,26-03-2020,16:33:00,BANK OF AMERICA CORPOR,NSY,1,22.445,USD,-22.45,USD,-20.39,EUR,1.1010,-0.50,EUR,-20.89,EUR


In [5]:
stocks = Transactions['Produkt'].unique()

In [6]:
closed_positions = Transactions[Transactions['Liczba'] < 0]
current_positions = Transactions[Transactions['Liczba'] > 0]

In [7]:
closed_positions

,Data,Czas,Produkt,Giełda referenc,Liczba,Kurs,waluta_pozycji,Wartość lokalna,waluta_wartości_lok,Wartość,waluta_zak,Kurs wymian,Opłata transakcyjna,Opł_trans_waluta,Razem,waluta_zak_razem
33,09-11-2020,16:56:00,DELTA AIR LINES INC.,NSY,-2,35.1810,USD,70.36,USD,59.53,EUR,1.1820,-0.51,EUR,59.02,EUR
46,12-06-2020,16:27:00,ISHRC S&P 500,LSE,-1,307.0100,USD,307.01,USD,271.96,EUR,1.1289,-2.08,EUR,269.88,EUR
47,12-06-2020,16:26:00,VANGUARD S&P500,LSE,-1,46.2175,GBP,46.22,GBP,51.48,EUR,0.8978,-2.02,EUR,49.46,EUR
49,05-06-2020,18:24:00,SANOFI ADR,NDQ,-1,50.2000,USD,50.20,USD,44.46,EUR,1.1290,-0.50,EUR,43.96,EUR
54,01-06-2020,19:58:00,VANDA PHARMACEUTICALS,NDQ,-2,11.5800,USD,23.16,USD,20.81,EUR,1.1128,-0.51,EUR,20.30,EUR
55,01-06-2020,19:57:00,AIKIDO PHARMA INC,NDQ,-100,0.7100,USD,71.00,USD,63.80,EUR,1.1128,-0.86,EUR,62.94,EUR
57,26-05-2020,00:00:00,TARONIS TECHNOLOGIES INC. - COMMON STCOK,NDQ,-100,0.0000,USD,0.00,USD,0.00,EUR,1.0840,NaN,NaN,0.00,EUR
59,18-05-2020,20:22:00,ARENA PHARMACEUTICALS,NDQ,-1,52.9600,USD,52.96,USD,48.53,EUR,1.0913,-0.50,EUR,48.03,EUR
61,28-04-2020,17:15:00,BANK OF AMERICA CORPOR,NSY,-1,24.1500,USD,24.15,USD,22.28,EUR,1.0838,-0.50,EUR,21.78,EUR
67,16-04-2020,21:36:00,TARONIS TECHNOLOGIES INC. - COMMON STCOK,NDQ,-150,0.2265,USD,33.98,USD,31.34,EUR,1.0842,-1.05,EUR,30.29,EUR


In [7]:
def parse_stock(stock):
    stock_url = 'https://query1.finance.yahoo.com/v7/finance/download/{}?'
    params = {
         'range' : '2y',
        'interval' : '1d',
        'events' : 'history'
    }
    response_stock = requests.get(stock_url.format(stock), params = params,headers={'User-agent': 'Mozilla/5.0'})
    file_stock = StringIO(response_stock.text)
    reader_stock = csv.reader(file_stock)
    data_stock = pd.DataFrame (reader_stock, columns = ['Data','Open','High','Low','Close','Adj Close','Volume']).drop([0])
    data_stock['Data'] = pd.to_datetime(data['Data'], yearfirst=True).dt.date
    data_stock['Adj Close'] = pd.to_numeric(data['Adj Close'])
    data_stock['Close'] = pd.to_numeric(data['Close'])
    return data_stock

In [95]:
def parse_currency(currency = 'EUR=X'):
    currency_url = 'https://query1.finance.yahoo.com/v7/finance/download/{}?'
    params = {
         'range' : '2y',
        'interval' : '1d',
        'events' : 'history'
    }
    response_currency = requests.get(currency_url.format(currency), params = params,headers={'User-agent': 'Mozilla/5.0'})
    file_currency = StringIO(response_currency.text)
    reader_currency = csv.reader(file_currency)
    fx = pd.DataFrame (reader_currency, columns = ['Data','Open','High','Low','Close','Adj Close','Volume']).drop([0])
    fx['Data'] = pd.to_datetime(fx['Data'], yearfirst=True).dt.date
    fx['Adj Close'] = pd.to_numeric(fx['Adj Close'])
    fx['Close'] = pd.to_numeric(fx['Close'])
    return fx

In [75]:
def profit(stock):
    initial_time_point = data_stock[data_stock['Data'] == stock.iloc[0][0]]
    amount_of_bought_stocks = stock.iloc[0,4]
    sum_of_bought_stocks = (-1)*stock.iloc[0,9]*amount_of_bought_stocks
    average_price = sum_of_bought_stocks/amount_of_bought_stocks
    roi_brutto = []
    first_day_gain = ((data_stock.iloc[initial_time_point.index[0],5]*fx.iloc[initial_time_point.index[0],5]) - average_price)/average_price
    roi_brutto.append(first_day_gain)
    for i in range (initial_time_point.index[0],len(data_stock)):
        for j in range(len(stock)):
            if data_stock.iloc[i][0] == stock.iloc[j][0]:
                amount_of_bought_stocks += stock.iloc[j,4]
                sum_of_bought_stocks += (-1)*stock.iloc[j,9] * stock.iloc[j,4]
        average_price = sum_of_bought_stocks/amount_of_bought_stocks
        following_gains = ((data_stock.iloc[i,5]*fx.iloc[i,5]) - average_price)/average_price
        roi_brutto.append(ret)
    return roi_brutto
